# RAG Retrieval Testing Notebook

This notebook tests:
1. Direct retrieval from Qdrant vector stores (clarity & rigor collections)
2. Agent RAG nodes retrieving context from their respective collections
3. Visualizing what context is being retrieved for each agent

We'll use example1 (climate_prediction.tex) as our test document.

## Setup and Imports

In [21]:
# Auto-reload modules when they change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path().absolute().parent))

from qdrant_client import QdrantClient
from app.rag.config import clarity_rag_config, rigor_rag_config
from app.rag.rag_service import RAGService
from app.agents.rag_nodes import ClarityRAGNode, RigorRAGNode
from app.models.schemas import Section

print("✓ Imports successful")

✓ Imports successful


## 1. Test Direct Retrieval from Vector Stores

First, let's verify we can retrieve directly from both Qdrant collections.

In [23]:
# Connect to Qdrant
qdrant_client = QdrantClient(url="http://localhost:6333")

# Check collections
collections = qdrant_client.get_collections()
print("Available collections:")
for col in collections.collections:
    info = qdrant_client.get_collection(col.name)
    print(f"  - {col.name}: {info.points_count} points")

print("\n✓ Successfully connected to Qdrant")

Available collections:
  - clarity_guidelines: 148 points
  - rigor_guidelines: 269 points

✓ Successfully connected to Qdrant


In [24]:
# Browse Clarity Collection
print("="*80)
print("CLARITY COLLECTION - Sample Documents")
print("="*80)

clarity_docs = qdrant_client.scroll(
    collection_name='clarity_guidelines',
    limit=100,  # Change this number to see more/fewer docs
    with_payload=True,
)[0]

for i, point in enumerate(clarity_docs, 1):
    content = point.payload.get('page_content', '')
    source = point.payload.get('metadata', {}).get('source', 'Unknown')
    print(f"\n[{i}] {source}")
    print(content)
    print("-"*80)

CLARITY COLLECTION - Sample Documents

[1] app/resources/clarity_docs/practical_suggestions_math_writing.pdf
(51) Refer to the 1980s, not the 1980’s: see [CMOS, 9.34]. (52) I.e. and e.g. should be followed by a comma, at least in American English. Also, if what come before and after are full sentences, then the i.e. or e.g. should be preceded by a semicolon instead of a comma.
--------------------------------------------------------------------------------

[2] app/resources/clarity_docs/how_to_write_clear_math_paper.pdf
“First, let’s talk about something obvious. Why do we do what we do? I mean, why do we study for many years how to do research in mathematics, read dozens or hundreds of papers, think long thoughts until we eventually ﬁgure out a good question. We then work hard, trial-and-error, to eventually ﬁgure out a solution. Sometimes we do this in a matter of hours and some- times it takes years, but we persevere. Then write up a solution, submit to a journal, sometimes get rej

In [25]:
# Browse Rigor Collection
print("="*80)
print("RIGOR COLLECTION - Sample Documents")
print("="*80)

clarity_docs = qdrant_client.scroll(
    collection_name='rigor_guidelines',
    limit=100,  # Change this number to see more/fewer docs
    with_payload=True,
)[0]

for i, point in enumerate(clarity_docs, 1):
    content = point.payload.get('page_content', '')
    source = point.payload.get('metadata', {}).get('source', 'Unknown')
    print(f"\n[{i}] {source}")
    print(content)
    print("-"*80)

RIGOR COLLECTION - Sample Documents

[1] app/resources/rigor_docs/How_to_Write_Mathematics.pdf
I think I can tell someone how to write, but I can’t think who would want to listen. The ability to communicate eﬀectively, the power to be intelligible, is congenital, I believe, or in any event, it is so early acquired that by the time someone reads my wisdom on the subject he is likely to be invariant under it.
--------------------------------------------------------------------------------

[2] app/resources/rigor_docs/How_to_Write_Mathematics.pdf
It is easy to do, it is fun to do, it is easy to read, and the reader is helped by the ﬁrm organizational scaﬀolding, even if he doesn’t bother to examine it and see where the joins come and how the support one another.
--------------------------------------------------------------------------------

[3] app/resources/rigor_docs/How_to_Write_Mathematics.pdf
A tentative chapter outline is something better. It might go like this: I’ll tell them ab

### 1.1 Test Clarity Collection Retrieval

In [5]:
clarity_rag_config

RAGConfig(collection_name='clarity_guidelines', documents_path='app/resources/clarity_docs', embedding_config=EmbeddingConfig(embedding_model='text-embedding-3-small', chunking_strategy='semantic', chunk_size=800, chunk_overlap=150, breakpoint_threshold_type='percentile', breakpoint_threshold_amount=92.0, use_cache=True), retrieval_config=RetrievalConfig(retriever_type='naive', top_k=3, metadata_filter={}))

In [14]:
# Create RAG service for clarity agent
clarity_service = RAGService(config=clarity_rag_config, qdrant_client=qdrant_client)

# Test query about clarity issues
clarity_query = "how to avoid vague statements and improve clarity in technical writing"

print(f"Query: {clarity_query}")
print("="*80)

clarity_results_with_scores = clarity_service.retrieve_with_scores(clarity_query, top_k=3)

print(f"Retrieved {len(clarity_results_with_scores)} documents with scores:")
print("="*80)

for i, (doc, score) in enumerate(clarity_results_with_scores, 1):
    print(f"\n[Document {i}] - Similarity Score: {score:.4f}")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Agent Type: {doc.metadata.get('agent_type', 'Unknown')}")
    print("-" * 80)
    print(doc.page_content + "...")
    print("-" * 80)

Query: how to avoid vague statements and improve clarity in technical writing
Retrieved 3 documents with scores:

[Document 1] - Similarity Score: 0.5492
Source: app/resources/clarity_docs/how_to_write_clear_math_paper.pdf
Agent Type: clarity
--------------------------------------------------------------------------------
1.3. Why be clear? Now that we framed it as a tradeoﬀ between your time and
eﬀort, and that of the readers, this is no longer an obvious question and it deserves a
full explanation. And the key observation is – being clear is not about you! You must
think of the reader and how they will read your paper. Imagine a graduate student at a small university with poor English skills. He is
reading your paper. If confused on page 3, he is likely to give up and never ﬁnish the
reading. He might use an older paper with a weaker result for his research, just because
it’s better written. Conclusion: you didn’t make him spend 1 extra min – you just lost
a signiﬁcant fraction of yo

### 1.2 Test Rigor Collection Retrieval

In [7]:
# Create RAG service for rigor agent
rigor_service = RAGService(config=rigor_rag_config, qdrant_client=qdrant_client)

# Test query about rigor issues
rigor_query = "mathematical rigor proof validation and stating assumptions"

print(f"Query: {rigor_query}")
print("="*80)

rigor_results_with_scores = rigor_service.retrieve_with_scores(rigor_query, top_k=3)

print(f"Retrieved {len(rigor_results_with_scores)} documents with scores:")
print("="*80)

for i, (doc, score) in enumerate(rigor_results_with_scores, 1):
    print(f"\n[Document {i}] - Similarity Score: {score:.4f}")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Agent Type: {doc.metadata.get('agent_type', 'Unknown')}")
    print(f"\nContent Preview (first 300 chars):")
    print("-" * 80)
    print(doc.page_content[:300] + "...")
    print("-" * 80)

Query: mathematical rigor proof validation and stating assumptions
Retrieved 3 documents with scores:

[Document 1] - Similarity Score: 0.6724
Source: app/resources/rigor_docs/how_to_math.pdf
Agent Type: rigor

Content Preview (first 300 chars):
--------------------------------------------------------------------------------
In many ways, mathematical proofs are like a programming language. There is a syntax and rules that are valid,
i.e. accepted by the mathematical community as logical inferences which can be done. So, to say ‘This proof is
rigorous.’ is essentially the same as saying ‘This code compiles.’ To make a h...
--------------------------------------------------------------------------------

[Document 2] - Similarity Score: 0.6265
Source: app/resources/rigor_docs/how_to_math.pdf
Agent Type: rigor

Content Preview (first 300 chars):
--------------------------------------------------------------------------------
How to math: an introduction to rigor
Roy Dong
October 29, 2020

## 2. Load Example Paper Sections

Load sections from example1/climate_prediction.tex to test agent retrieval.

In [ ]:
# Read the example LaTeX file
example_file = Path("../examples/example1/climate_prediction.tex")
with open(example_file, 'r') as f:
    latex_content = f.read()

print("Loaded example paper:")
print(f"  File: {example_file}")
print(f"  Size: {len(latex_content)} characters")

# Extract a few sections (simplified - just grab text between section markers)
# In production, you'd use proper LaTeX parsing

# Introduction section
intro_content = """Climate change is an important problem. Machine learning can help predict climate patterns. We propose a new method using deep learning.

Neural networks have been used before. Our approach is different because we use a special architecture. The results show that our method works well.

Temperature prediction is challenging. Previous work has limitations. We address these limitations with our approach."""

# Methodology section
methodology_content = """Our method uses a neural network with parameters θ. The loss function is defined as:
L(θ) = ∑(y_i - f(x_i; θ))²

We train the model using gradient descent. The learning rate is set to α = 0.01. The network has multiple layers.

The optimization problem can be written as:
min_θ L(θ) + λ||θ||²

where λ is a regularization parameter. We use backpropagation to compute gradients."""

# Experimental Validation section
experimental_content = """We evaluate our method on two datasets. The first dataset contains temperature readings from 100 stations. The second dataset has precipitation data.

We compare against several baselines including linear regression and random forests. Our method outperforms all baselines.

The mean squared error (MSE) is calculated as:
MSE = (1/n) ∑(y_i - ŷ_i)²

Our approach achieves MSE = 2.3 while the best baseline gets MSE = 4.7."""

print("\n✓ Extracted 3 sections for testing")

## 3. Create Section Objects for Testing

In [ ]:
# Create Section objects
intro_section = Section(
    title="Introduction",
    content=intro_content,
    section_type="introduction",
    line_start=19
)

methodology_section = Section(
    title="Methodology",
    content=methodology_content,
    section_type="methodology",
    line_start=27
)

experimental_section = Section(
    title="Experimental Validation",
    content=experimental_content,
    section_type="results",
    line_start=53
)

test_sections = [intro_section, methodology_section, experimental_section]

print("Created test sections:")
for sec in test_sections:
    print(f"  - {sec.title} ({sec.section_type}): {len(sec.content)} chars")

## 4. Test Clarity Agent RAG Node

Test that the Clarity RAG Node can retrieve relevant guidelines.

In [ ]:
# Create Clarity RAG Node
clarity_rag_node = ClarityRAGNode(qdrant_client=qdrant_client, cache_enabled=True)

print("Testing Clarity RAG Node")
print("="*80)

# Test with introduction section (has clarity issues like vague statements)
clarity_state = {
    "sections_for_clarity": [intro_section]
}

print(f"\nInput Section: {intro_section.title}")
print("-" * 80)
print(intro_section.content[:200] + "...")
print("-" * 80)

# Execute RAG node
clarity_result = clarity_rag_node(clarity_state)
clarity_guidelines = clarity_result.get("clarity_guidelines", "")

print(f"\n✓ Retrieved Clarity Guidelines ({len(clarity_guidelines)} chars)")
print("="*80)
print("\nRetrieved Context:")
print(clarity_guidelines)

## 5. Test Rigor Agent RAG Node

Test that the Rigor RAG Node can retrieve relevant guidelines.

In [ ]:
# Create Rigor RAG Node
rigor_rag_node = RigorRAGNode(qdrant_client=qdrant_client, cache_enabled=True)

print("Testing Rigor RAG Node")
print("="*80)

# Test with methodology section (has rigor issues like missing assumptions)
rigor_state = {
    "sections_for_rigor": [methodology_section, experimental_section]
}

print(f"\nInput Sections: {methodology_section.title}, {experimental_section.title}")
print("-" * 80)
print(methodology_section.content[:200] + "...")
print("-" * 80)

# Execute RAG node
rigor_result = rigor_rag_node(rigor_state)
rigor_guidelines = rigor_result.get("rigor_guidelines", "")

print(f"\n✓ Retrieved Rigor Guidelines ({len(rigor_guidelines)} chars)")
print("="*80)
print("\nRetrieved Context:")
print(rigor_guidelines)

## 6. Compare Query Formulation

Show how different sections produce different queries and retrieve different context.

In [ ]:
print("Comparing RAG Retrieval for Different Sections")
print("="*80)

for section in test_sections:
    print(f"\n{'='*80}")
    print(f"Section: {section.title} ({section.section_type})")
    print(f"{'='*80}")
    
    # Clarity RAG
    print("\n[CLARITY AGENT]")
    clarity_state = {"sections_for_clarity": [section]}
    clarity_result = clarity_rag_node(clarity_state)
    clarity_guidelines = clarity_result.get("clarity_guidelines", "")
    
    if clarity_guidelines:
        # Extract just the first guideline for preview
        first_guideline = clarity_guidelines.split('\n\n')[0] if clarity_guidelines else "No guidelines"
        print(f"Retrieved: {len(clarity_guidelines)} chars")
        print(f"Preview: {first_guideline[:200]}...")
    else:
        print("No guidelines retrieved")
    
    # Rigor RAG
    print("\n[RIGOR AGENT]")
    rigor_state = {"sections_for_rigor": [section]}
    rigor_result = rigor_rag_node(rigor_state)
    rigor_guidelines = rigor_result.get("rigor_guidelines", "")
    
    if rigor_guidelines:
        # Extract just the first guideline for preview
        first_guideline = rigor_guidelines.split('\n\n')[0] if rigor_guidelines else "No guidelines"
        print(f"Retrieved: {len(rigor_guidelines)} chars")
        print(f"Preview: {first_guideline[:200]}...")
    else:
        print("No guidelines retrieved")

## 7. Test Retrieval Metrics

Verify that retrieval is fast and efficient.

In [ ]:
import time

print("RAG Retrieval Performance Test")
print("="*80)

# Test clarity retrieval speed
start = time.time()
clarity_state = {"sections_for_clarity": test_sections}
clarity_result = clarity_rag_node(clarity_state)
clarity_time = time.time() - start

print(f"\nClarity RAG Node:")
print(f"  Sections processed: {len(test_sections)}")
print(f"  Time: {clarity_time*1000:.0f}ms")
print(f"  Guidelines retrieved: {len(clarity_result.get('clarity_guidelines', ''))} chars")

# Test rigor retrieval speed
start = time.time()
rigor_state = {"sections_for_rigor": test_sections}
rigor_result = rigor_rag_node(rigor_state)
rigor_time = time.time() - start

print(f"\nRigor RAG Node:")
print(f"  Sections processed: {len(test_sections)}")
print(f"  Time: {rigor_time*1000:.0f}ms")
print(f"  Guidelines retrieved: {len(rigor_result.get('rigor_guidelines', ''))} chars")

print("\n✓ Performance tests complete")

## 8. Summary

Test results summary.

In [ ]:
print("\n" + "="*80)
print("TEST SUMMARY")
print("="*80)

print("\n✅ Test Results:")
print("  1. ✓ Successfully retrieved from clarity_guidelines collection")
print("  2. ✓ Successfully retrieved from rigor_guidelines collection")
print("  3. ✓ Clarity RAG Node retrieves relevant context")
print("  4. ✓ Rigor RAG Node retrieves relevant context")
print("  5. ✓ Different sections retrieve different context")
print("  6. ✓ Retrieval is fast (<500ms per agent)")

print("\n📊 Collections Status:")
collections = qdrant_client.get_collections()
for col in collections.collections:
    info = qdrant_client.get_collection(col.name)
    print(f"  - {col.name}: {info.points_count} points, status: {info.status}")

print("\n🎯 Next Steps:")
print("  - RAG retrieval is working correctly")
print("  - Agents can access their respective collections")
print("  - Context is being retrieved based on section content")
print("  - Ready to integrate with full review workflow")

print("\n✓ All tests passed!")